# Multiple Artificial Neural Network

This script will attempt to create a MANN for the reconstruction problem of EIT. Each network will be trained to output the pixel value of a single element in the output image.

The network inputs are the 208 diffrenetial voltages obtained from the reconstrution problem. As there are 576 pixels in each image object's element data ( `img.elem_data` ) there will be 576 individual networks used to generate each of the pixel values.

## Network Architecture

**Trial1:** (208,10,10,1)


In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
in_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/input_data.csv', delimiter=',', prefix="Trial", skiprows=0, header=None)
out_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/output_data.csv', delimiter=',', prefix="Trial", skiprows=0, header=None)

input_data = in_df.values
output_data = out_df.values


In [4]:
# Number of networks
num_networks = 576

#Generic form of each network
l0 = tf.keras.layers.Dense(units=208, activation='elu', input_dim=208)  
l1 = tf.keras.layers.Dense(units=10, activation='elu')
l2 = tf.keras.layers.Dense(units=10, activation='elu')
l3 = tf.keras.layers.Dense(units=1)

#Create each model
arr = []
for x in tqdm(range(num_networks)):
  arr.append( tf.keras.Sequential([l0,l1,l2,l3]) )
  arr[x].compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1))
  
  


100%|██████████| 576/576 [00:22<00:00, 28.95it/s]


In [0]:
#Train the models
hist = []
for x in tqdm(range(num_networks)):
  hist.append( arr[x].fit(input_data, np.array([output_data[:,x]]).T, shuffle=True, verbose=0) )

 
print('Completed Training')

  2%|▏         | 9/576 [00:28<31:45,  3.36s/it]

In [81]:
# list = np.linspace(560,575,16)
# for x in list:
#   i = int(x)
#   print(x)
# arr[565].fit(input_data, np.array([output_data[:,565]]).T, shuffle=True, verbose = 0 )
arr[566].fit(input_data, np.array([output_data[:,560]]).T, shuffle=True, verbose = 0 )

# np.array([output_data[:,565]]).T.shape

In [0]:
def amalg_prediction(in_arr):
  